In [ ]:
import numpy as np
import pandas as pd

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [ ]:
adults = pd.read_csv('adult.csv')
adults = adults.replace("?", np.nan)


# Task 1
Print the count of men and women in the dataset.

In [1]:
adults.value_counts(["sex"])


# Task 2
Find the average age of men in dataset

In [ ]:
male_data = adults[adults["sex"] == "Male"]
avg_male = male_data["age"].mean()
avg_male


# Task 3
Get the percentage of people from Poland (native-country)

In [ ]:
adults["native-country"].unique()
poland_data = adults[adults["native-country"] == "Poland"]
percent_poland = (len(poland_data) / len(adults)) * 100
percent_poland


# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [2]:
high_salary = adults[adults["salary"] == ">50K"]
low_salary = adults[adults["salary"] == "<=50K"]

mean_high_salary_age = high_salary["age"].mean()
std_high_salary_age = high_salary["age"].std()

mean_low_salary_age = low_salary["age"].mean()
std_low_salary_age = low_salary["age"].std()


# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [3]:
higher_education = ["Bachelors", "Prof-school", "Assoc-acdm", "Assoc-voc", "Masters", "Doctorate"]
no_education = adults[~adults["education"].isin(higher_education)]
high_salary_no_education = high_salary[high_salary["education"].isin(no_education["education"])]
high_salary_no_education


# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [ ]:
adults.groupby("education")["age"].describe()


# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [5]:
married_adults = adults[adults["marital-status"].str.startswith("Married")]
not_married_adults = adults[~adults["marital-status"].str.startswith("Married")]

married_salary = (married_adults["salary"] == ">50K").sum()
not_married_salary = (not_married_adults["salary"] == ">50K").sum()

print(married_salary, not_married_salary)
print("Married earns more" if married_salary > not_married_salary else "Not married earns more")

categories = ["Married", "Not Married"]
counts = [married_salary, not_married_salary]


# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [6]:
max_hours = adults["hours-per-week"].max()
count_max_hours = (adults["hours-per-week"] == max_hours).sum()
hours_count = adults["hours-per-week"].value_counts().sort_index()


# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [ ]:
adults = pd.DataFrame(adults, columns=["age", "hours-per-week", "salary K$"])
correlation_matrix = adults.corr()
correlation_matrix

